# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [12]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy
import pathlib
import re
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import sklearn
import warnings
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ninen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ninen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ninen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ninen\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:
import flask
flask.__version__

'2.1.2'

In [2]:
# load data from database
engine = sqlalchemy.create_engine(r"sqlite:///C:\Users\ninen\Desktop\Datapipeline\data\DisasterResponse.db")
df = pd.read_sql_table('DisasterResponse',engine)

### 2. Write a tokenization function to process your text data

In [3]:
# def Tfid(texts):
#     """
#     TfidfVectorizer function

#     Args:
#         texts (List): List to vectorizer

#     Returns:
#         Dataframe : Dataframe of vectorized
#     """
#     vectorizer = TfidfVectorizer()
#     vectorized = vectorizer.fit_transform(texts)
#     return vectorized.toarray()


In [4]:
def tokenize(text):
    detected_urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LinearSVC()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
df = df[df.iloc[:,3:].sum(axis=1) != 0]

In [7]:
df.reset_index(drop=True,inplace=True)

In [8]:
X = df.iloc[:,:3]
Y = df.iloc[:,3:]

In [9]:
X = df['message'].values.tolist()

In [10]:
indexY = df.iloc[:,3:].columns[np.mean(Y,axis=0) != 0] 

In [11]:
Y = Y[indexY].values.tolist()

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2,shuffle=True)

In [13]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023728C35090>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=LinearSVC()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
def metrics(model,X_test,y_test):    
    Precision = []
    F1Score = []
    Recall = []
    from sklearn.metrics import precision_score,f1_score,recall_score
    for i in range(len(y_test[:50])):
        y_pred = model.predict(X_test)[i]
        Precision.append(precision_score(y_test[i],y_pred,average='weighted'))
        F1Score.append(f1_score(y_test[i],y_pred,average='weighted'))
        Recall.append(recall_score(y_test[i],y_pred,average='weighted'))
    npPrecision = np.array(Precision) 
    npF1Score = np.array(F1Score)
    npRecall = np.array(Recall)
    npPrecision = npPrecision[npPrecision > 0]
    npF1Score = npF1Score[npF1Score > 0]
    npRecall = npRecall[npRecall > 0]
    print("Precision",np.mean(npPrecision))
    print("Recall   ",np.mean(npRecall))
    print("F1Score  ",np.mean(npF1Score))
    return np.mean(npPrecision),np.mean(npRecall),np.mean(npF1Score)

In [20]:
metrics(pipeline,X_test,y_test)

KeyboardInterrupt: 

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
def partial(func, /, *args, **keywords):
          def newfunc(*fargs, **fkeywords):
              newkeywords = {**keywords, **fkeywords}
              return func(*args, *fargs, **newkeywords)
          newfunc.func = func
          newfunc.args = args
          newfunc.keywords = keywords
          return newfunc

In [16]:
import parfit.parfit as pf
from sklearn.model_selection import ParameterGrid
parameters = ParameterGrid({
    'clf__estimator__penalty':['l2'],
    'clf__estimator__loss':['hinge','squared_hinge'],
    'clf__estimator__tol':[1e-3,1e-4,1e-5],
    'clf__estimator__C':[1,2,3],
    'clf__estimator__verbose': [0,1],
    'clf__estimator__max_iter': [10000,50000]
})
best_model, best_score, all_models, all_scores = pf.bestFit(pipeline, parameters, 
     X_train, y_train, X_test, y_test, scoreLabel='F1')
print(best_model, best_score)

-------------FITTING MODELS-------------


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  49 out of  72 | elapsed:  2.8min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  57 out of  72 | elapsed:  3.6min remaining:   56.7s
[Parallel(n_jobs=-1)]: Done  65 out of  72 | elapsed:  4.2min remaining:   27.2s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  5.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


-------------SCORING MODELS-------------


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  6.1min


Too many dimensions to plot.
Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024305A89F30>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=LinearSVC(C=1, loss='hinge',
                                                           max_iter=10000,
                                                           tol=0.001)))]) 0.9366987807869316


[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 14.8min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
clf = clf.best_estimator_

In [ ]:
clf.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize2 at 0x000001BCFC387B50>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=LinearSVC(C=1, loss='hinge', max_iter=50000,
                                             tol=0.001)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize2 at 0x000001BCFC387B50>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=LinearSVC(C=1, loss='hinge', max_iter=50000,
                                           tol=0.001)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__t

In [ ]:
metrics(clf,X_test,y_test)

C:\Users\ninen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ninen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ninen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precisio

Precision 0.9564943009547835
Recall    0.947142857142857
F1Score   0.9422430607974894


(0.9564943009547835, 0.947142857142857, 0.9422430607974894)

### 9. Export your model as a pickle file

In [ ]:
import pickle
pickle.dump(pipeline, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
pd.__version__

NameError: name 'pd' is not defined